In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import fitz

def fetch_text(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--accept-cookies")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")    
    driver = webdriver.Chrome(options=options)

    if url.lower().endswith('.pdf'):
        response = requests.get(url)
        response.raise_for_status()
        
        pdf_text = ""
        with fitz.open(stream=response.content, filetype="pdf") as pdf:
            for page_num in range(pdf.page_count):
                pdf_text += pdf[page_num].get_text("text")
        raw_text = pdf_text

    else:
        driver.get(url)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        raw_text = soup.get_text(separator=' ')
    
    cleaned_text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑüÜ0-9\s,.]', '', raw_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("....", "")
    return cleaned_text.strip()


def generate_dataframe(urls):
    data = []
    for url in urls:
        text = fetch_text(url)
        if text:
            data.append({"url": url, "text": text})
    
    df = pd.DataFrame(data, columns=["url", "text"])
    return df

urls = ["https://smowl.net/es/",
        "https://smowl.net/es/proctoring-empresa/",
        "https://smowl.net/es/soporte-para-examinados/",
        "https://smowl.net/es/proctoring-educacion/",
        "https://smowl.net/es/productos-de-proctoring/",
        "https://smowl.net/es/sobre-nosotros/",
        "https://smowl.net/es/faqs-proctoring/",
        "https://smowl.net/es/faqs-proctoring/usuario/",
        "https://smowl.net/es/soporte-para-examinados/",
        "https://media.smowltech.net/html/privacy/dpa/smowltech_dpa_es.html",
        "https://media.smowltech.net/html/conditions/conditions_SCM_2_es.html",
        "https://media.smowltech.net/html/conditions/conditions_GDPR_2_es.html",
        "https://media.smowltech.net/html/terms/conditions_SMOWLPLUS_2_es.html",
        "https://smowl.net/en/privacy-policy/",
        "https://media.smowltech.net/pdf/guides/smowlcm_userguide_installation_es.pdf"]

df = generate_dataframe(urls)

In [14]:
df

,url,text
0,https://smowl.net/es/,SMOWL Software de supervisión de exámenes onli...
1,https://smowl.net/es/proctoring-empresa/,Proctoring para empresas Smowl Consent Details...
2,https://smowl.net/es/soporte-para-examinados/,Soporte técnico Smowl Skip to content mySmowlt...
3,https://smowl.net/es/proctoring-educacion/,Supervisar exámenes online Smowl Consent Detai...
4,https://smowl.net/es/productos-de-proctoring/,Software de proctoring SMOWL Proctoring Sistem...
5,https://smowl.net/es/sobre-nosotros/,"Smowltech, quiénes somos Consent Details IABV2..."
6,https://smowl.net/es/faqs-proctoring/,Preguntas frecuentes SMOWL Skip to content myS...
7,https://smowl.net/es/faqs-proctoring/usuario/,Información y soporte para usuarios SMOWL Skip...
8,https://smowl.net/es/soporte-para-examinados/,Soporte técnico Smowl Skip to content mySmowlt...
9,https://media.smowltech.net/html/privacy/dpa/s...,Smowltech DPA CONTRATO DE ENCARGADO DE TRATAMI...


In [15]:
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

model_id = "intfloat/multilingual-e5-large"

model = SentenceTransformer(model_id)
embeddings = model.encode(df["text"])

df['embeddings'] = embeddings.tolist()
loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=model.max_seq_length,
    chunk_overlap=model.max_seq_length //10,
)
docs = text_splitter.split_documents(documents)

In [16]:
from langchain.vectorstores import FAISS
from langchain_core.embeddings import Embeddings


class SentenceTransformerEmbeddings(Embeddings):
    """Embedding model using Sentence Transformers."""

    def __init__(self, model_name: str = ''):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """Embed a list of documents."""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text: str) -> list[float]:
        """Embed a single query text."""
        return self.model.encode(text, convert_to_numpy=True).tolist()

vectorstore_faiss = FAISS.from_documents(
    documents=docs,
    embedding=SentenceTransformerEmbeddings(model_id)
)

In [17]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Qué es el CM?",k=6)
[i[0].page_content for i in matching_docs]

['Soporte Blog Solicitar demo gratis Solicitar demo gratis Preguntas frecuentes SMOWL Somos transparentes y te damos información detallada Si necesitas hacernos una consulta que no esté en esta página puedes contactar directamente con nuestro equipo de soporte técnico. Contacta aquí Información para instructores Información para instructores Información para usuarios Información para usuarios Qué es SMOWL CM E s una aplicación de escritorio que supervisa la actividad de un usuario durante una prueba online.',
 'desde el enlace proporcionado por nuestra institución. Actualización de CMPuede que una vez que termines con la instalación de CM, o antes de iniciar tu examen o actividad, aparezca una ventana indicando que el sistema se encuentra buscando nuevas actualizaciones de la herramienta.En caso de que existan, te será notificado con el mensaje Actualización disponible.Para actualizar tu versión de SMOWLCM, deberás descargarla y posteriormente instalarla pulsando sobre los botones corr

In [18]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Quien es el CEO de smowl?",k=10)
[i[0].page_content for i in matching_docs]

['europea y por ello cumplimos totalmente con el Reglamento General de Protección de Datos. Además, como recibimos fondos por parte de la Unión Europea, cumplimos con las exigencias de ética y seguridad. Es el momento de la transparencia Sabemos que elegir la herramienta adecuada es importante para ti. Por eso, te damos una demostración de SMOWL y atendemos todas tus preguntas en una reunión para ampliar la información que necesites. Solicitar mi demo gratuita o mi reunión personalizada Sobre Smowltech',
 'Preguntas frecuentes SMOWL Skip to content mySmowltech ES EN Usos Empresa Educación Planes Nosotros Soporte Preguntas frecuentes Estudiante Soporte técnico Blog Menu Usos Empresa Educación Planes Nosotros Soporte Preguntas frecuentes Estudiante Soporte técnico Blog Usos Empresa Educación Planes Smowltech Soporte Preguntas frecuentes Estudiante Soporte Blog Solicitar demo gratis Menu Usos Empresa Educación Planes Smowltech Soporte Preguntas frecuentes Estudiante Soporte Blog Solicitar

In [19]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Como solicito una prueba?",k=10)
[i[0].page_content for i in matching_docs]

['exámenes online Instalación sencilla e intuitiva. Soporte técnico 247 para incidencias. Privacidad del usuario garantizada cumpliendo la RGPD. Supervisión 100 personalizable y escalable. Solicitar una demo gratis 25 licencias gratis Ver una demo grabada Ahorra hasta un 75 en exámenes Más de 200 instituciones en todo el mundo confían en nosotros 8,8 Satisfacción de los usuarios con nuestro soporte Casos prácticos Nuestros clientes te lo cuentan Ver casos de estudio Mientras el alumno realiza el examen,',
 'Una vez que nos escribas te responderemos en el menor tiempo posible por correo electrónico o videollamada. Comunica una incidencia Nuestro producto está basado en la tecnología. Es minucioso y eficaz, pero hasta las máquinas a veces fallan. Si te encuentras alguna anomalía, contacta con nosotros. Pregunta o pide información Tienes alguna duda Compártela con el equipo. Pregúntanos cualquier aspecto relacionado con el funcionamiento de la herramienta. Para contactar con nuestro equip

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="llama3.1:8b")

In [39]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """You will be acting as a customer support chatbot named Smowlbot, created by the company Smowltech, a leader in online proctoring solutions.  
Your goal is to provide support to customers regarding Smowltech’s products and services, ensuring secure and reliable environments for remote exams and assessments.  
You are part of Smowltech's customer support team. Always stay in character.

Here are some important rules for the interaction:  
- Use the chat history and the following pieces of context to answer the question.  
- Only use relevant parts of the context.  
- If the context is meaningful to answer the query, ensure your response aligns with the context and refrain from fabricating answers.  
- Do not include web, email, or other links unless they are in the context.  
- Provide assistance, recommendations, or advice only on matters related to Smowltech's proctoring technology, services, and processes.  
- Always maintain a friendly and professional tone in your responses.  
- Use three sentences maximum and keep the answer clear, concise, and easy to follow.  
- If the question is unclear or you need additional crucial user information to provide an accurate answer, feel free to ask the user for more details.  

Here is the context:  
<context>  
{context}  
</context>  
Here is the user’s question:  
<question>  
{question}  
</question>  

How do you respond to the user’s question?  
Answer in the language of the question.  
Always focus on responding to the user's messages and using only the context that directly relates to their queries.  
Put your response in <response></response> tags.  
Think about your answer first before you respond.  
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 7}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [40]:
qa({"query": "Quien es el CEO de smowl?"})

{'query': 'Quien es el CEO de smowl?',
 'result': '<response>\n¡Hola! El CEO de Smowltech es Ricardo Vea, según lo mencionado en la sección "Quiénes somos" del contexto. Espero que esto te ayude. ¿Necesitas algo más? \n</response>',
 'source_documents': [Document(metadata={'url': 'https://smowl.net/es/proctoring-educacion/', 'embeddings': [0.009212099015712738, -0.024286828935146332, -0.04283325374126434, -0.04123631492257118, 0.028068577870726585, -0.016986100003123283, -0.01795998029410839, 0.059627726674079895, 0.054913267493247986, -0.03468724340200424, 0.04671730101108551, 0.014280407689511776, -0.02414078824222088, 0.014591476880013943, -0.024885717779397964, -0.033449623733758926, -0.007777397520840168, 0.014239219017326832, -0.0017299804603680968, -0.010292072780430317, 0.02773328870534897, -0.00836335401982069, -0.0674254447221756, -0.024320419877767563, -0.0018389482283964753, -0.0077667636796832085, -0.029554415494203568, -0.03924396261572838, -0.010303492657840252, -0.02907

In [41]:
qa({"query": "Que es smowl CM?"})

{'query': 'Que es smowl CM?',
 'result': '<response>\nSmowl CM es una aplicación de escritorio que supervisa la actividad de un usuario durante una prueba online. Es una herramienta integral de nuestro sistema de supervisión en línea, diseñada para garantizar la integridad y seguridad de los exámenes y evaluaciones remotos.\n</response>',
 'source_documents': [Document(metadata={'url': 'https://smowl.net/es/faqs-proctoring/', 'embeddings': [-0.02227167598903179, -0.018223633989691734, 0.009959639981389046, -0.04109868034720421, 0.036626093089580536, -0.026952851563692093, -0.0636819377541542, 0.09104368090629578, 0.06051824986934662, -0.021146712824702263, 0.032423194497823715, 0.021860133856534958, -0.02857307717204094, -0.009389901533722878, -0.016490865498781204, -0.014434000477194786, -0.0175913218408823, 0.037578798830509186, 0.02706814557313919, -0.028948908671736717, 0.01761258952319622, -0.014790868386626244, -0.02205779030919075, -0.029013467952609062, 0.019498148933053017, -0

In [42]:
qa({"query": "Que LMS soporta smowl?"})

{'query': 'Que LMS soporta smowl?',
 'result': '<response>\nSmowl se integra en las principales plataformas educativas como Moodle, Canvas, Open LMS y Blackboard. Además, trabajamos con más plataformas educativas para ofrecer una integración versátil y sencilla. Puedes encontrar más detalles en nuestra sección de FAQs sobre la integración con LMS. </response>',
 'source_documents': [Document(metadata={'url': 'https://smowl.net/es/proctoring-empresa/', 'embeddings': [0.00421610102057457, -0.03589024022221565, -0.026134513318538666, -0.03612004593014717, 0.029110020026564598, -0.01800394244492054, -0.03876907005906105, 0.0675753802061081, 0.06213271617889404, -0.031981125473976135, 0.05276601389050484, 0.02355881966650486, -0.0104226591065526, 0.005290317349135876, -0.025382576510310173, -0.027557820081710815, -0.020535197108983994, 0.011888666078448296, 0.009618612937629223, -0.00864478200674057, 0.03135007619857788, -0.013470003381371498, -0.06927747279405594, -0.02116689644753933, -0.

In [43]:
qa({"query": "Que debo hacer si me voy a examinar?"})

{'query': 'Que debo hacer si me voy a examinar?',
 'result': '<response>\nSi te vas a examinar, te recomendamos revisar la Guía de utilización que sugerimos contactarte con tu empresa o institución para solicitarla. Además, asegúrate de tener instalado y funcionando correctamente el sistema de proctoring SMOWL, ya que esto es crucial para garantizar un proceso de evaluación seguro y justo. ¡Buena suerte en tu examen! \n</response>',
 'source_documents': [Document(metadata={'url': 'https://smowl.net/es/productos-de-proctoring/', 'embeddings': [0.006279048044234514, -0.030562078580260277, -0.03287860006093979, -0.03648426756262779, 0.0346144437789917, -0.020031016319990158, -0.033142391592264175, 0.07799594849348068, 0.05147446319460869, -0.03401864320039749, 0.04878762736916542, 0.02133100852370262, -0.01772461086511612, 0.010823599994182587, -0.03477185592055321, -0.028035225346684456, -0.020619813352823257, 0.0191243514418602, 0.002837433945387602, -0.013265581801533699, 0.02187860570

In [44]:
qa({"query": "Me duele la cabeza, me tomo un hibuprofeno?"})

{'query': 'Me duele la cabeza, me tomo un hibuprofeno?',
 'result': 'I cannot provide a response that focuses on medical advice, especially pain relief medication. If you would like to discuss Smowltech’s proctoring solutions or have questions related to our services, I would be happy to assist. Alternatively, if you are experiencing issues with our platform, feel free to describe the problem and I will do my best to help.',
 'source_documents': [Document(metadata={'url': 'https://smowl.net/es/proctoring-empresa/', 'embeddings': [0.00421610102057457, -0.03589024022221565, -0.026134513318538666, -0.03612004593014717, 0.029110020026564598, -0.01800394244492054, -0.03876907005906105, 0.0675753802061081, 0.06213271617889404, -0.031981125473976135, 0.05276601389050484, 0.02355881966650486, -0.0104226591065526, 0.005290317349135876, -0.025382576510310173, -0.027557820081710815, -0.020535197108983994, 0.011888666078448296, 0.009618612937629223, -0.00864478200674057, 0.03135007619857788, -0.01

In [45]:
file_name = "../faiss_index"
vectorstore_faiss.save_local(file_name)